In [30]:
import requests
import pandas as pd
import numpy as np
import json
import os
from dotenv import load_dotenv, find_dotenv


dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

True

In [3]:
FACEIT_PLAYERID = os.environ.get("FACEIT_PLAYERID")
KEY = os.environ.get("KEY")
HEADER = {"Authorization": f"Bearer {KEY}"}
BASIC_URL = "https://open.faceit.com/data/v4/"

In [4]:

#used to retreive time registered and elo
def retreive_player_by_id(id: str) -> dict:
    response = shorter_url(url=f"players/{id}")
    return response
# used to shorted the URL and not to specify header every time
def shorter_url(url = "") -> dict:
    request = BASIC_URL + url
    return requests.get(request, headers = HEADER).json()

#gets number of games for paginaition
def get_number_of_games_played(id: str) -> int:
    response = int(shorter_url(url = f"players/{id}/stats/csgo")["lifetime"]["Matches"])
    return response

#retreived time registered
def get_time_registered(id: str) -> str:
    response = retreive_player_by_id(id)
    return response["activated_at"]


#retreived elo and level
def get_level_elo(id: str) -> int:
    response = retreive_player_by_id(id)
    return (response["games"]["csgo"]["skill_level"], response["games"]["csgo"]["faceit_elo"])


#retreived all stats from all matches
def retreive_player_game_stats(id: str, offset = 0, limit = 100) -> dict:
    games_played = get_number_of_games_played(id)
    offset = 0
    times_to_run = games_played // limit
    response_array = []

    while times_to_run > 0:
        response = shorter_url(url = f"players/{id}/games/csgo/stats?offset={offset}&limit={limit}")["items"]
        response_array.extend(response)
        times_to_run-=1
        offset+=limit
        
    else:
        response = shorter_url(url = f"players/{id}/games/csgo/stats?offset={offset}&limit={limit}")["items"]
        response_array.extend(response)
    return response_array


# use for myself once only
def retreive_player_history(id: str, offset = 0, limit = 100) -> dict:
    games_played = get_number_of_games_played(id)
    registered = get_time_registered(id)
    registered = int(pd.Timestamp(registered).timestamp())
    offset = 0
    times_to_run = games_played // limit
    response_array = []

    while times_to_run > 0:
        response = shorter_url(url = f"players/{id}/history?game=csgo&from={registered}&offset={offset}&limit={limit}")["items"]
        response_array.extend(response)
        times_to_run-=1
        offset+=limit
        
    else:
        response = shorter_url(url = f"players/{id}/history?game=csgo&from={registered}&offset={offset}&limit={limit}")["items"]
        response_array.extend(response[1::])
    return response_array




In [9]:
# MatchHistory = retreive_player_history(FACEIT_PLAYERID)
# with open(f"../data/raw/{FACEIT_PLAYERID}_match_history.json", "w") as file:
#     json.dump(MatchHistory, file )   

In [6]:
# GamesStats = retreive_player_game_stats(FACEIT_PLAYERID)
# with open(f"../data/raw/{FACEIT_PLAYERID}_games_stats.json", "w") as file:
#     json.dump(GamesStats, file )

In [51]:
def get_all_players_id():
    with open(f"../data/raw/{FACEIT_PLAYERID}_match_history.json", "r") as file:
        # df = pd.json_normalize(file, max_level=4)
        obj = json.load(file)
        list_2D = (list(map(lambda x: x["playing_players"], obj)))
        list_1D = {item for sub_list in list_2D for item in sub_list}

    return list_1D

In [52]:
players_IDs = get_all_players_id()

In [55]:
sample = list(players_IDs)[0:10]

In [63]:
def download_player_data(id: str) -> None:
    data = retreive_player_game_stats(id)
    with open(f"../data/raw/{id}_match_history.json", "w") as file:
        json.dump(data, file)

In [64]:
for id in sample:
    download_player_data(id)

In [88]:
with open("../data/raw/Game Stats/40476548-200a-4e2b-8db2-0d8c98765efc_games_stats.json", "r") as file:
    df = json.load(file)

In [89]:
df = pd.json_normalize(df)

In [90]:
def rename_cols(text: str):
    text = text.split(".")[1]
    text = text.split(" ")
    text = "".join(list(map(str.capitalize, text)))
    return text

df.rename(columns=rename_cols, inplace=True)

In [92]:
df.columns

Index(['Region', 'CreatedAt', 'FinalScore', 'UpdatedAt', 'K/dRatio',
       'MatchRound', 'Kills', 'QuadroKills', 'TripleKills', 'Result',
       'PentaKills', 'Mvps', 'FirstHalfScore', 'BestOf', 'Map', 'Game',
       'MatchId', 'Deaths', 'Rounds', 'SecondHalfScore', 'GameMode', 'Score',
       'Nickname', 'PlayerId', 'OvertimeScore', 'CompetitionId', 'Headshots%',
       'Team', 'K/rRatio', 'Headshots', 'Winner', 'Assists'],
      dtype='object')

In [93]:
df["OvertimeScore"]

0    0
1    0
2    0
Name: OvertimeScore, dtype: object

In [91]:
columns_to_keep_games_stats = ["PlayerId", "CreatedAt", "Kills", "Deaths", "Assists", "Headshots", "FirstHalfScore", "SecondHalfScore",'QuadroKills', 'TripleKills', 'PentaKills', 'Mvps', "OvertimeScore", "Rounds", "Score", "Result"]

,Region,CreatedAt,FinalScore,UpdatedAt,K/dRatio,MatchRound,Kills,QuadroKills,TripleKills,Result,...,Nickname,PlayerId,OvertimeScore,CompetitionId,Headshots%,Team,K/rRatio,Headshots,Winner,Assists
0,EU,2017-08-14T07:24:37.965Z,1,2017-08-14T07:24:57.654Z,0.62,1,5,0,0,0,...,Andryxa_aga,40476548-200a-4e2b-8db2-0d8c98765efc,0,,60,team__RED_1488,0.5,3,d189dcb2-8bd9-4b6c-9b0f-783ba385a697,0
1,EU,2017-07-30T21:28:56.944Z,6,2017-07-30T21:29:12.899Z,0.61,1,11,0,1,0,...,Andryxa_aga,40476548-200a-4e2b-8db2-0d8c98765efc,0,,18,team_zokuuu01,0.5,2,e9bc2ad4-d3e5-4093-91e3-eeb3095f5c49,0
2,EU,2017-07-30T20:25:29.367Z,10,2017-07-30T20:25:44.065Z,0.53,1,9,0,0,0,...,Andryxa_aga,40476548-200a-4e2b-8db2-0d8c98765efc,0,,11,team_debczukm,0.35,1,66ee92f5-600e-4cf1-a3d1-f62af86cd85a,4
